In [4]:
!pip install matplotlib
!pip install unlzw
!pip install pyarc

  Created wheel for unlzw: filename=unlzw-0.1.1-cp37-cp37m-linux_x86_64.whl size=21441 sha256=9aa16bb1cb4dacca7429a0ddd161ae8551d48d681000c302f3a9e6fd01d8e13c
  Stored in directory: /root/.cache/pip/wheels/6e/36/71/0956632d0fc14c99edc168063b7ff06c3462e19855a0df36db
Successfully built unlzw
     |████████████████████████████████| 357 kB 39.2 MB/s 
  Created wheel for pyfim: filename=pyfim-6.28-cp37-cp37m-linux_x86_64.whl size=537774 sha256=430da3a6d63c37fd7d21bf67e0ecb3dad321d09ec62ca74387b5eb14508da962
  Stored in directory: /root/.cache/pip/wheels/08/9f/26/09cb4efd027e46f96e0a0f33d0a74be614d3caf89c1eeb75a8
Successfully built pyfim


In [5]:
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from unlzw import unlzw
import csv, re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data Preprocessing

In [9]:
# Load the iris dataset
# iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
iris = pd.read_csv('drive/MyDrive/iris.data', header = None)
iris.columns=["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Class"]
iris.to_csv("iris.csv", index = None, header = True)
#iris = pd.read_csv('iris.csv',index= None, header = True)

#Load the waveform dataset #CART decision tree algorithm: 72% (Information from the .names file)
with open('drive/MyDrive/waveform.data.Z', 'rb') as fh:
    compressed_data = fh.read()
    uncompressed_data = unlzw(compressed_data)
data = uncompressed_data.decode('utf-8').splitlines()
with open("wave.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+',line))
wave = pd.read_csv('wave.csv', header = None)
wave.to_csv("wave.csv", index = None, header = True)
wave_labels = []
for i in range(21):
  wave_labels.append('x'+str(i))
wave_labels.append('class')
wave.columns=wave_labels
wave.to_csv("wave.csv", index = None, header = True)

#Load the breast cancer dataset
breastCancer = pd.read_csv("drive/MyDrive/breast-cancer.data")
breastCancer.columns=["class","age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]
breastCancer.to_csv("breastCancer.csv", index = None, header = True)
#wave.to_csv("breastCancer.csv", index = None, header = True)


#Load the german dataset
german = pd.read_csv('drive/MyDrive/german.data', header = None)
german.head()
german_columns=["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker","goodBad"]
german = german[0].str.split(" ", n = 20, expand = True)
german.columns = german_columns
german.to_csv("german.csv", index = None, header = True)
#german.columns = german_columns

#Load the heart dataset
heart = pd.read_csv('drive/MyDrive/heart.dat', header = None)
heart = heart[0].str.split(" ", n = 13, expand = True)
heart.columns=["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal","class"]
heart_columns= ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]
heart.to_csv("heart.csv", index = None, header = True)


#Load the kaggle dataset
#stroke
stroke = pd.read_csv("drive/MyDrive/CZ4032DAM/datasets/healthcare-dataset-stroke-data.csv", sep=";", index_col=0)

#Heart Attack
campusPlacement = pd.read_csv("drive/MyDrive/CZ4032DAM/datasets/Placement_Data_Full_Class.csv", sep=";", index_col=0)


#2) Implement an algorithm of Classification based on Association rules. You can implement any algorithm in the paper, or a variant of these algorithms

In [ ]:
#TODO: split the train and test properly
data_train = pd.read_csv("iris.csv")
data_test = pd.read_csv("iris.csv")

txns_train = TransactionDB.from_DataFrame(data_train)
txns_test = TransactionDB.from_DataFrame(data_test)

# get the best association rules
rules = top_rules(txns_train.string_representation)

# convert them to class association rules
cars = createCARs(rules)

#classifier = M1Algorithm(cars, txns_train).build()
classifier = M2Algorithm(cars, txns_train).build()

iris_accuracy = classifier.test_transactions(txns_test)
print(iris_accuracy)

0.9933333333333333
